In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import *
import mysql.connector


In [2]:

cnx = mysql.connector.connect(user='spirz', 
                              password='this_is_my_PASSWORD_m8',
                              host='localhost',
                              database='dem_from_contours_lines',
                              autocommit=True)
cursor = cnx.cursor(buffered=True,dictionary=True)
contours_lines_table_name='contours_lines'
grid_nodes_table_name="grid_nodes"
grid_edges_table_name="grid_edges"
intersections_table_name="intersections_grid"
2

2

In [3]:
DELETE_TABLES=True
if DELETE_TABLES:
    cmd="""DROP TABLE IF EXISTS `%s`"""%intersections_table_name
    cursor.execute(cmd)
    cmd="""DROP TABLE IF EXISTS `%s`"""%grid_edges_table_name
    cursor.execute(cmd)
    cmd="""DROP TABLE IF EXISTS `%s`"""%grid_nodes_table_name
    cursor.execute(cmd)

2

2

In [4]:
CREATE_TABLES=True
if CREATE_TABLES:
    cmd="""CREATE TABLE `%s`(
    `index_x` INT NOT NULL,
    `index_y` INT NOT NULL,
    `geometry` POINT NOT NULL,
    `elevation` FLOAT,
    PRIMARY KEY (index_x,index_y)
    );
        """%grid_nodes_table_name
    
    cursor.execute(cmd)

    cmd="""CREATE TABLE `%s`(
    `index_x_begin` INT NOT NULL,
    `index_y_begin` INT NOT NULL,
    `index_x_end` INT NOT NULL,
    `index_y_end` INT NOT NULL,
    `length` FLOAT NOT NULL,
    `intersects` BOOLEAN,


    FOREIGN KEY (index_x_begin,index_y_begin) REFERENCES %s(index_x,index_y),
    FOREIGN KEY (index_x_end,index_y_end) REFERENCES %s(index_x,index_y),
    PRIMARY KEY (index_x_begin,index_y_begin,index_x_end,index_y_end)

    );
        """%(grid_edges_table_name,grid_nodes_table_name,grid_nodes_table_name)
    
    cursor.execute(cmd)

    

    cmd="""CREATE TABLE `%s`(
        `id` INT NOT NULL AUTO_INCREMENT,
        `index_x_begin` INT NOT NULL,
        `index_y_begin` INT NOT NULL,
        `index_x_end` INT NOT NULL,
        `index_y_end` INT NOT NULL,
        `edge_coordinate` FLOAT NOT NULL,
        `geometry` POINT NOT NULL,
        `elevation` FLOAT NOT NULL,

        FOREIGN KEY (index_x_begin,index_y_begin,index_x_end,index_y_end) REFERENCES %s(index_x_begin,index_y_begin,index_x_end,index_y_end),
        UNIQUE (index_x_begin,index_y_begin,index_x_end,index_y_end,edge_coordinate),
        PRIMARY KEY (id)

          )
          ENGINE=InnoDB;
        """%(intersections_table_name,grid_edges_table_name)
    cursor.execute(cmd)

2

2

In [5]:
t1=time.time()
place_name='Corse, France'
area=ox.geocode_to_gdf(place_name)
crs=area.estimate_utm_crs()
area=area.to_crs(crs)
t2=time.time()
print(t2-t1)


0.5657474994659424


In [6]:
step,simplify_coeff=50,1000
poly=area.iloc[0]['geometry']
t1=time.time()
coords=np.array(poly.boundary.coords)
x_min,x_max,y_min,y_max=np.min(coords[:,0]),np.max(coords[:,0]),np.min(coords[:,1]),np.max(coords[:,1])
X,Y=np.arange(x_min,x_max,step),np.arange(y_min,y_max,step)
points_df=[{'index_x':i,'index_y':j,'geometry':Point(x,y)} for i,x in enumerate(X) for j,y in enumerate(Y)]
t2=time.time()
print(t2-t1)
points_df=gpd.GeoDataFrame(points_df,geometry='geometry',crs=crs)
t2=time.time()
print(t2-t1)
points_df=points_df[points_df['geometry'].within(poly.simplify(simplify_coeff))]
t2=time.time()
print(t2-t1)
points_df=points_df.set_index(pd.MultiIndex.from_tuples(zip(points_df['index_x'],points_df['index_y'])))
t2=time.time()
print(t2-t1)

44.890350341796875
58.69277501106262
242.5453281402588
245.09686017036438


In [ ]:
t1=time.time()
edges_df=[]
for i,j in points_df.index:
    if (i+1,j) in points_df.index:
        edges_df.append({'begin':(i,j),'end':(i+1,j),'geometry':LineString([points_df.loc[i,j]['geometry'],points_df.loc[i+1,j]['geometry']])})
    if (i,j+1) in points_df.index:
        edges_df.append({'begin':(i,j),'end':(i,j+1),'geometry':LineString([points_df.loc[i,j]['geometry'],points_df.loc[i,j+1]['geometry']])})
    
    
t2=time.time()
print(t2-t1)    
edges_df=gpd.GeoDataFrame(edges_df,geometry='geometry',crs=crs)
t2=time.time()
print(t2-t1)
edges_df=edges_df.set_index(np.arange(len(edges_df)))
edges_df['id_edge']=edges_df.index
edges_df['edge']=edges_df.apply(lambda row:(row['begin'],row['end']),axis=1)
edges_df['length']=step
t2=time.time()
print(t2-t1)

In [ ]:
t1=time.time()
intersection=increasing_elevations_intersections(edges_df,crs,cursor)
t2=time.time()
print('total : %f'%(t2-t1))

In [ ]:
ADD_GRID_DATA=True
if ADD_GRID_DATA:
    t1=time.time()
    edges=edges_df['edge']
    points_df=points_df.to_crs('epsg:4326')
    
    
    string_list=',\n'.join(['(%i,%i,ST_GeomFromText(\'%s\'))'%(index_x,index_y,row['geometry'].wkt) for (index_x,index_y),row in points_df.iterrows()])
    cmd="INSERT IGNORE INTO %s(index_x,index_y,geometry) VALUES %s;"%(grid_nodes_table_name,string_list)
    cursor.execute(cmd)

    
    intersecting_edges=intersection['edge'].unique()
    string_list=',\n'.join(['(%i,%i,%i,%i,%f,TRUE)'%(edge[0][0],edge[0][1],edge[1][0],edge[1][1],step) for edge in intersecting_edges])
    cmd="INSERT INTO %s(index_x_begin,index_y_begin,index_x_end,index_y_end,length,intersects) VALUES %s;"%(grid_edges_table_name,string_list)
    cursor.execute(cmd)

    inside_edges=set(edges).difference(intersecting_edges)

    string_list=',\n'.join(['(%i,%i,%i,%i,%f,FALSE)'%(edge[0][0],edge[0][1],edge[1][0],edge[1][1],step) for edge in inside_edges])
    cmd="INSERT INTO %s(index_x_begin,index_y_begin,index_x_end,index_y_end,length,intersects) VALUES %s;"%(grid_edges_table_name,string_list)
    cursor.execute(cmd)

    intersection=intersection.to_crs('epsg:4326')

    string_list=',\n'.join(['(%i,%i,%i,%i,%f,%f,ST_GeomFromText(\'%s\'))'%(row['edge'][0][0],row['edge'][0][1],row['edge'][1][0],row['edge'][1][1],row['edge_coordinate'],row['elevation'],row['geometry'].wkt)  for _,row in intersection.iterrows()])
    cmd="INSERT INTO %s (index_x_begin,index_y_begin,index_x_end,index_y_end,edge_coordinate,elevation,geometry) VALUES %s;"%(intersections_table_name,string_list)
    cursor.execute(cmd)
    
    t2=time.time()
    print(t2-t1)
2